In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model

In [5]:
dataset_ = pd.read_csv('/content/drive/Shareddrives/Đồ án đa ngành/Flood_Forecast/sample_data.csv')

In [6]:
dataset_.head(10)

,timestamp,rainfall,tide,flooded
0,2010-01-06 00:00:00,0.066667,-0.370,49808.19372
1,2010-01-06 01:00:00,0.100000,-1.340,50388.62414
2,2010-01-06 02:00:00,0.000000,-1.825,50938.57857
3,2010-01-06 03:00:00,0.033333,-1.902,51456.76400
4,2010-01-06 04:00:00,0.000000,-1.498,51945.07580
5,2010-01-06 05:00:00,0.000000,-0.823,52405.46288
6,2010-01-06 06:00:00,0.000000,-0.175,52840.08047
7,2010-01-06 07:00:00,0.000000,0.433,53251.67338
8,2010-01-06 08:00:00,0.000000,1.153,53701.17872
9,2010-01-06 09:00:00,0.000000,1.449,55593.49608


In [11]:
dataset1 = dataset_.values
dataset = dataset1[:,1:]
print(dataset.shape)
dataset

(23989, 3)


array([[0.066666667, -0.37, 49808.19372],
       [0.1, -1.34, 50388.62414],
       [0.0, -1.825, 50938.57857],
       ...,
       [0.0, -0.15, 6156285.693],
       [0.0, -1.097, 6133080.269],
       [0.0, -2.118, 6108817.302]], dtype=object)

In [14]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset_transform = scaler.fit_transform(dataset)

In [15]:
def prepare_data(dataset, step_size=10): 
  data_X = []
  for i in range(len(dataset)-step_size-1):
	  a = dataset[i:(i+step_size), :]
	  data_X.append(a)
  return np.array(data_X, dtype=np.float32)

In [16]:
pregen_data = prepare_data(dataset_transform)

In [18]:
pregen_data.shape

(23978, 10, 3)

In [19]:
model = load_model('/content/drive/Shareddrives/Đồ án đa ngành/Flood_Forecast/model.h5')

In [28]:
gen_data = model.predict(pregen_data)

In [29]:
gen_data

array([[-0.00413   ,  0.6878141 ,  0.01505017],
       [-0.00626494,  0.5790888 ,  0.01608501],
       [-0.00421675,  0.47943497,  0.01526983],
       ...,
       [-0.00829193,  0.6576503 ,  0.253953  ],
       [-0.00793962,  0.483796  ,  0.25214556],
       [-0.00449033,  0.3226655 ,  0.25170103]], dtype=float32)

In [30]:
gen_data = scaler.inverse_transform(gen_data)

In [31]:
gen_data

array([[-3.6032740e-02,  1.2372007e+00,  4.2756378e+05],
       [-5.4659314e-02,  5.4038012e-01,  4.5353803e+05],
       [-3.6789659e-02, -9.8301291e-02,  4.3307712e+05],
       ...,
       [-7.2344102e-02,  1.0438807e+00,  6.4239655e+06],
       [-6.9270343e-02, -7.0351429e-02,  6.3785990e+06],
       [-3.9176494e-02, -1.1030366e+00,  6.3674410e+06]], dtype=float32)

In [41]:
a = gen_data

In [42]:
a[:,0][a[:,0] < 0] = 0.0

In [44]:
gen_data[:,0][gen_data[:,0] < 0] = 0

In [46]:
gen_data

array([[ 0.0000000e+00,  1.2372007e+00,  4.2756378e+05],
       [ 0.0000000e+00,  5.4038012e-01,  4.5353803e+05],
       [ 0.0000000e+00, -9.8301291e-02,  4.3307712e+05],
       ...,
       [ 0.0000000e+00,  1.0438807e+00,  6.4239655e+06],
       [ 0.0000000e+00, -7.0351429e-02,  6.3785990e+06],
       [ 0.0000000e+00, -1.1030366e+00,  6.3674410e+06]], dtype=float32)

In [47]:
df = pd.DataFrame(gen_data, columns = ['rainfall','tide','flooded'])

In [48]:
df.head()

,rainfall,tide,flooded
0,0.000000,1.237201,427563.78125
1,0.000000,0.540380,453538.03125
2,0.000000,-0.098301,433077.12500
3,0.000000,-0.963651,419133.09375
4,0.041949,-1.447201,347234.21875


In [49]:
df.to_csv('/content/drive/Shareddrives/Đồ án đa ngành/Flood_Forecast/data_gen1.csv')